In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import os
import cv2
import random
import time
import tensorflow as tf

from detector import Detector

In [ ]:
MODEL_PATH = 'model.pb'
IMAGES_FOLDER = '/home/dan/datasets/COCO/images/val2017/'

# Load the model

In [ ]:
detector = Detector(MODEL_PATH)

# Get an image

In [ ]:
names = os.listdir(IMAGES_FOLDER)

In [ ]:
n = random.choice(names)
path = os.path.join(IMAGES_FOLDER, n)

image = Image.open(path)
image = image.resize((640, 640))
image

# Predict

In [ ]:
outputs = detector(np.array(image), score_threshold=0.0001)

# Show detections

In [ ]:
EDGES = [
    (0, 1), (0, 2),
    (1, 3), (2, 4),
    (5, 7), (7, 9), (6, 8), (8, 10),
    (11, 13), (13, 15), (12, 14), (14, 16),
    (3, 5), (4, 6),
    (5, 11), (6, 12)
]


In [ ]:
outputs['keypoint_positions'][1]

In [ ]:
def draw_everything(image, outputs):

    image_copy = image.copy()
    image_copy.putalpha(255)
    draw = ImageDraw.Draw(image_copy, 'RGBA')
    width, height = image_copy.size
    scaler = np.array([height, width, height, width])
    
    n = outputs['num_boxes']
    boxes = scaler * outputs['boxes']
    for i, box in enumerate(boxes):
        ymin, xmin, ymax, xmax = box
        draw.rectangle([(xmin, ymin), (xmax, ymax)], outline='red')
       
        keypoints = outputs['keypoint_positions'][i]
        keypoints = keypoints[:, [1, 0]]
        keypoints *= np.array([xmax - xmin, ymax - ymin])
        keypoints += np.array([xmin, ymin])
        keypoints = np.concatenate([keypoints, np.ones([17, 1])], axis=1)

        for (p, q) in EDGES:

            x1, y1, v1 = keypoints[p]
            x2, y2, v2 = keypoints[q]

            both_visible = v1 > 0 and v2 > 0
            if both_visible:
                draw.line([(x1, y1), (x2, y2)])

        for j in range(17):
            x, y, v = keypoints[j]
            if v > 0:
                s = 8
                draw.ellipse([
                    (x - s, y - s),
                    (x + s, y + s)
                ], fill='red')
    
#     mask = outputs['segmentation_masks']
#     m, M = mask.min(), mask.max()
#     mask = (mask - m)/(M - m)
#     mask = np.expand_dims(mask, 2)
#     color = np.array([255, 255, 255])
#     mask = Image.fromarray((mask*color).astype('uint8'))
#     mask.putalpha(mask.convert('L'))
#     mask = mask.resize((width, height))
#     image_copy.alpha_composite(mask)
    return image_copy

In [ ]:
draw_everything(image, outputs)

In [ ]:
segmentation_masks = results['segmentation_masks'][0]
segmentation_masks = results['segmentation_masks'][0]

In [ ]:
m = segmentation_masks.min()
M = segmentation_masks.max()
segmentation_masks = (segmentation_masks - m)/(M - m)
segmentation_masks = Image.fromarray((segmentation_masks*255.0).astype('uint8'))
segmentation_masks

# Measure speed

In [ ]:
times = []
for _ in range(110):
    start = time.perf_counter()
    boxes, labels, scores = detector(image_array, score_threshold=0.25)
    times.append(time.perf_counter() - start)
    
times = np.array(times)
times = times[10:]
print(times.mean(), times.std())